### This notebook coves:
* Creating and Using Advanced Index Structures
* Slicing, Sorting, Swapping, Shuffling, and Pivoting Index levels
* Cross sections, label and position based selection
* A closer look at internals of MultiIndex/Hierarchical Index

### Revision:
* Creating MultiIndex: 
    - pd.read_csv(url, index_col=['a_col', 'b_col'])
    - df.set_index(['a_col', 'b_col'], inplace=True)
    - pd.Dataframe(data=data, index=['a_col', 'b_col']
* Indexing in MultiIndex:
    - df.loc['first_index']
    - df.loc['first_index','second_index'], df.loc['first_index','second_index'].close
    - df.loc[('first_index','second_index'), ['close','high']]
    - df.loc[['first_index1','first_index2'], ['close','high']]
    - df.loc[(['first_index1','first_index2'], ['second_index1', 'second_index2']), ['close','high']]
    - df.loc[['first_index':'first_index2'], 'close':'high']
    - df.loc[(slice('first_index1', 'first_index2'), slice('second_index1', 'second_index2'), slice(None)), 'close':'high']
    - df.loc[pd.SliceIndex['first_index1':'first_index2', 'second_index1':'second_index2',:], :]
    - df.xs('FB', level=1)
    - df.xs(('2018-01-02','FB'), level=(0,1), drop_level=False, axis=0)
* MultiIndex anatomy:
    - df.index.names
    - df.index.nlevels
    - df.index.levels
    - df.index.levshape
    - df.index.values
* Shuffling and Removing Index:
    - df.swaplevel(1,2), df.swaplevel('a_index', 'b_index')
    - df.reorder_levels([2,1,3,0]), df.reorder_levels(['b_index', 'c_index', 'k_index', 'a_index'])
    - df.droplevel(1), df.droplevel([1,2])
    - df.reset_index(level=1/[1,2], drop=False)
* Sorting and other methods for MultiIndex:
    - df.sort_index(inplace=False)
    - df.sortlevels(0, ascending=False, sort_remaining=True)
    - df.is_lexsorted() = df.is_monotonic_increasing
    - df.index.to_flat_index() - works only on index, values doesn't change
    - df.index.set_names([a_col, b_col, c_col], inplace=True)
* stack() and unstack():
    - stack() makes series from df by making columns an index
    - unstack() makes inner most index of multiindex into columns
    - unstack(level=1, fill_value='-')
* Creating MultiIndex:
    - pd.MultiIndex.fromproduct([[a_col, b_col], [c_col, d_col]], names=['Volume', 'Ticker'])
* Transpose:
    - tech.set_index(['Trading Date', 'Volume Category']).transpose()

In [3]:
import pandas as pd
import numpy as np

In [4]:
tech_url = 'https://andybek.com/pandas-tech'
# 5 years of stock prices: amazon, google, microsoft, facebook, apple
tech = pd.read_csv(tech_url)

In [5]:
tech

,date,month,year,day,name,open,close,high,low,volume,volume_type
0,2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,medium
1,2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,low
2,2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,medium
3,2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,medium
4,2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,medium
...,...,...,...,...,...,...,...,...,...,...,...
7100,2019-08-23,8,2019,23,MSFT,137.19,133.39,138.35,132.80,38515386,medium
7101,2019-08-23,8,2019,23,AAPL,209.43,202.64,212.05,201.00,46882843,medium
7102,2019-08-23,8,2019,23,GOOGL,1185.17,1153.58,1195.67,1150.00,1813141,medium
7103,2019-08-23,8,2019,23,AMZN,1793.03,1749.62,1804.90,1745.23,5277898,medium


In [6]:
tech.year.value_counts()/5

2014    252.0
2015    252.0
2016    252.0
2017    251.0
2018    251.0
2019    163.0
Name: year, dtype: float64

In [7]:
tech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7105 entries, 0 to 7104
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7105 non-null   object 
 1   month        7105 non-null   int64  
 2   year         7105 non-null   int64  
 3   day          7105 non-null   int64  
 4   name         7105 non-null   object 
 5   open         7105 non-null   float64
 6   close        7105 non-null   float64
 7   high         7105 non-null   float64
 8   low          7105 non-null   float64
 9   volume       7105 non-null   int64  
 10  volume_type  7105 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 610.7+ KB


### Index and RangeIndex

In [8]:
# RnageIndex inherits from Index
print(type(tech.index), type(tech.columns))

<class 'pandas.core.indexes.range.RangeIndex'> <class 'pandas.core.indexes.base.Index'>


In [9]:
tech.set_index('date').loc['2014-01-22']

,month,year,day,name,open,close,high,low,volume,volume_type
date,,,,,,,,,,
2014-01-22,1,2014,22,MSFT,36.26,35.93,36.32,35.75,21965266,medium
2014-01-22,1,2014,22,FB,58.80,57.51,59.31,57.10,61495880,high
2014-01-22,1,2014,22,GOOGL,583.30,582.51,583.95,579.43,1573846,medium
2014-01-22,1,2014,22,AMZN,408.00,404.54,408.06,402.00,2069759,low
2014-01-22,1,2014,22,AAPL,78.70,78.79,79.61,78.26,13602762,low


### Creating MultiIndex

In [10]:
tech.set_index(['date','name'], inplace=True)

In [11]:
tech.head(10)

month  year  day    open   close    high     low    volume  \
date       name                                                                
2014-01-02 FB         1  2014    2   54.86   54.71   55.22   54.19  43257622   
           AAPL       1  2014    2   79.38   79.02   79.58   78.86   8398851   
           GOOGL      1  2014    2  557.73  556.56  558.88  554.13   1822719   
           MSFT       1  2014    2   37.35   37.16   37.40   37.10  30643745   
           AMZN       1  2014    2  398.80  397.97  399.36  394.02   2140246   
2014-01-03 FB         1  2014    3   55.00   54.56   55.65   54.53  38287706   
           GOOGL      1  2014    3  557.50  552.50  558.47  552.47   1669229   
           MSFT       1  2014    3   37.20   36.91   37.22   36.60  31134795   
           AAPL       1  2014    3   79.00   77.28   79.10   77.20  14043410   
           AMZN       1  2014    3  398.29  396.44  402.71  396.22   2213512   

                 volume_type  
date       name               
2014-01-02 FB         medium  
           AAPL          low  
           GOOGL      medium  
           MSFT       medium  
           AMZN       medium  
2014-01-03 FB         medium  
           GOOGL      medium  
           MSFT       medium  
           AAPL          low  
           AMZN       medium

In [12]:
# with read_csv

In [13]:
pd.read_csv(tech_url, index_col=['date','name'])

month  year  day     open    close     high      low  \
date       name                                                          
2014-01-02 FB         1  2014    2    54.86    54.71    55.22    54.19   
           AAPL       1  2014    2    79.38    79.02    79.58    78.86   
           GOOGL      1  2014    2   557.73   556.56   558.88   554.13   
           MSFT       1  2014    2    37.35    37.16    37.40    37.10   
           AMZN       1  2014    2   398.80   397.97   399.36   394.02   
...                 ...   ...  ...      ...      ...      ...      ...   
2019-08-23 MSFT       8  2019   23   137.19   133.39   138.35   132.80   
           AAPL       8  2019   23   209.43   202.64   212.05   201.00   
           GOOGL      8  2019   23  1185.17  1153.58  1195.67  1150.00   
           AMZN       8  2019   23  1793.03  1749.62  1804.90  1745.23   
           FB         8  2019   23   180.84   177.75   183.13   176.66   

                    volume volume_type  
date       name                         
2014-01-02 FB     43257622      medium  
           AAPL    8398851         low  
           GOOGL   1822719      medium  
           MSFT   30643745      medium  
           AMZN    2140246      medium  
...                    ...         ...  
2019-08-23 MSFT   38515386      medium  
           AAPL   46882843      medium  
           GOOGL   1813141      medium  
           AMZN    5277898      medium  
           FB     17331221      medium  

[7105 rows x 9 columns]

In [14]:
# another approach
# pd.DataFrame(data,index=['date','name'])

### Indexing in hierarchical Index:

In [15]:
tech.loc['2014-01-22']

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
MSFT,1,2014,22,36.26,35.93,36.32,35.75,21965266,medium
FB,1,2014,22,58.80,57.51,59.31,57.10,61495880,high
GOOGL,1,2014,22,583.30,582.51,583.95,579.43,1573846,medium
AMZN,1,2014,22,408.00,404.54,408.06,402.00,2069759,low
AAPL,1,2014,22,78.70,78.79,79.61,78.26,13602762,low


In [16]:
tech.loc['2014-01-22', 'AAPL']

month                 1
year               2014
day                  22
open               78.7
close             78.79
high              79.61
low               78.26
volume         13602762
volume_type         low
Name: (2014-01-22, AAPL), dtype: object

In [17]:
tech.loc['2014-01-22', 'AAPL'].close

78.79

In [18]:
tech.loc[('2014-01-22', 'AAPL'), 'close']

78.79

In [19]:
# iloc - agnostic to MultiIndex
# loc - cares

In [20]:
tech.head()

month  year  day    open   close    high     low    volume  \
date       name                                                                
2014-01-02 FB         1  2014    2   54.86   54.71   55.22   54.19  43257622   
           AAPL       1  2014    2   79.38   79.02   79.58   78.86   8398851   
           GOOGL      1  2014    2  557.73  556.56  558.88  554.13   1822719   
           MSFT       1  2014    2   37.35   37.16   37.40   37.10  30643745   
           AMZN       1  2014    2  398.80  397.97  399.36  394.02   2140246   

                 volume_type  
date       name               
2014-01-02 FB         medium  
           AAPL          low  
           GOOGL      medium  
           MSFT       medium  
           AMZN       medium

In [21]:
tech.iloc[2,4]

556.56

In [22]:
tech.loc[('2014-01-02', 'AAPL'), ['open', 'close']]

open     79.38
close    79.02
Name: (2014-01-02, AAPL), dtype: float64

In [23]:
tech.loc[('2014-01-02', 'AAPL'), 'open':'close']

open     79.38
close    79.02
Name: (2014-01-02, AAPL), dtype: float64

In [24]:
tech.iloc[1,3:5]

open     79.38
close    79.02
Name: (2014-01-02, AAPL), dtype: float64

In [25]:
tech.loc[['2014-01-02','2014-01-03']]

month  year  day    open   close    high     low    volume  \
date       name                                                                
2014-01-02 FB         1  2014    2   54.86   54.71   55.22   54.19  43257622   
           AAPL       1  2014    2   79.38   79.02   79.58   78.86   8398851   
           GOOGL      1  2014    2  557.73  556.56  558.88  554.13   1822719   
           MSFT       1  2014    2   37.35   37.16   37.40   37.10  30643745   
           AMZN       1  2014    2  398.80  397.97  399.36  394.02   2140246   
2014-01-03 FB         1  2014    3   55.00   54.56   55.65   54.53  38287706   
           GOOGL      1  2014    3  557.50  552.50  558.47  552.47   1669229   
           MSFT       1  2014    3   37.20   36.91   37.22   36.60  31134795   
           AAPL       1  2014    3   79.00   77.28   79.10   77.20  14043410   
           AMZN       1  2014    3  398.29  396.44  402.71  396.22   2213512   

                 volume_type  
date       name               
2014-01-02 FB         medium  
           AAPL          low  
           GOOGL      medium  
           MSFT       medium  
           AMZN       medium  
2014-01-03 FB         medium  
           GOOGL      medium  
           MSFT       medium  
           AAPL          low  
           AMZN       medium

In [26]:
tech.loc[(['2014-01-02','2014-01-03'], ['FB','AAPL']), :]

month  year  day   open  close   high    low    volume  \
date       name                                                           
2014-01-02 FB        1  2014    2  54.86  54.71  55.22  54.19  43257622   
           AAPL      1  2014    2  79.38  79.02  79.58  78.86   8398851   
2014-01-03 FB        1  2014    3  55.00  54.56  55.65  54.53  38287706   
           AAPL      1  2014    3  79.00  77.28  79.10  77.20  14043410   

                volume_type  
date       name              
2014-01-02 FB        medium  
           AAPL         low  
2014-01-03 FB        medium  
           AAPL         low

In [27]:
tech.loc['2014-01-02':'2014-01-06', 'open':'low']

open   close    high     low
date       name                                 
2014-01-02 FB      54.86   54.71   55.22   54.19
           AAPL    79.38   79.02   79.58   78.86
           GOOGL  557.73  556.56  558.88  554.13
           MSFT    37.35   37.16   37.40   37.10
           AMZN   398.80  397.97  399.36  394.02
2014-01-03 FB      55.00   54.56   55.65   54.53
           GOOGL  557.50  552.50  558.47  552.47
           MSFT    37.20   36.91   37.22   36.60
           AAPL    79.00   77.28   79.10   77.20
           AMZN   398.29  396.44  402.71  396.22
2014-01-06 MSFT    36.86   36.13   36.89   36.11
           AMZN   396.13  393.63  397.00  388.42
           AAPL    76.78   77.70   78.11   76.23
           GOOGL  556.51  558.66  559.43  553.22
           FB      54.39   57.20   57.26   54.05

In [28]:
tech.loc[('2014-01-02':'2014-01-06', 'AAPL'), 'open':'low']

SyntaxError: invalid syntax (2129925073.py, line 1)

### slice() object

In [29]:
tech.loc[(slice('2014-01-02','2014-01-06'), 'AAPL'), 'open':'low']

,,open,close,high,low
date,name,,,,
2014-01-02,AAPL,79.38,79.02,79.58,78.86
2014-01-03,AAPL,79.00,77.28,79.10,77.20
2014-01-06,AAPL,76.78,77.70,78.11,76.23


In [30]:
# OPEN PRICES OR FB AND AAPL FOR ALL DATES
tech.loc[(slice(None), ['FB', 'AAPL']), 'open']

date        name
2014-01-02  FB       54.86
2014-01-03  FB       55.00
2014-01-06  FB       54.39
2014-01-07  FB       57.67
2014-01-08  FB       57.59
                     ...  
2019-08-19  AAPL    210.62
2019-08-20  AAPL    210.88
2019-08-21  AAPL    212.99
2019-08-22  AAPL    213.19
2019-08-23  AAPL    209.43
Name: open, Length: 2842, dtype: float64

### pd.IndexSlice()

In [31]:
# high and low for all days for amzn and aapl.
tech.loc[pd.IndexSlice[:,['AMZN','AAPL']],'high':'low']

,,high,low
date,name,,
2014-01-02,AMZN,399.36,394.02
2014-01-03,AMZN,402.71,396.22
2014-01-06,AMZN,397.00,388.42
2014-01-07,AMZN,398.47,394.29
2014-01-08,AMZN,403.00,396.04
...,...,...,...
2019-08-19,AAPL,212.73,210.03
2019-08-20,AAPL,213.35,210.32
2019-08-21,AAPL,213.65,211.60


In [32]:
tech.loc[pd.IndexSlice['2014-01-06':'2014-01-22', ['AAPL', 'FB']], 'high':'low']

high    low
date       name              
2014-01-06 AAPL  78.11  76.23
           FB    57.26  54.05
2014-01-07 FB    58.55  57.22
           AAPL  77.99  76.85
2014-01-08 AAPL  77.94  76.96
           FB    58.41  57.23
2014-01-09 AAPL  78.12  76.48
           FB    58.96  56.65
2014-01-10 AAPL  77.26  75.87
           FB    58.30  57.06
2014-01-13 AAPL  77.50  75.70
           FB    58.25  55.38
2014-01-14 FB    57.78  56.10
           AAPL  78.10  76.81
2014-01-15 FB    58.57  57.27
           AAPL  80.03  78.81
2014-01-16 FB    58.02  56.83
           AAPL  79.55  78.81
2014-01-17 FB    57.82  56.07
           AAPL  78.87  77.13
2014-01-21 FB    58.58  56.50
           AAPL  78.58  77.20
2014-01-22 FB    59.31  57.10
           AAPL  79.61  78.26

### cross section with xs()

In [33]:
# xs() is a label based indexing

In [34]:
tech.xs('2018-01-02')

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
AMZN,1,2018,2,1172.00,1189.01,1190.00,1170.51,2694494,medium
GOOGL,1,2018,2,1053.02,1073.21,1075.98,1053.02,1588268,medium
FB,1,2018,2,177.68,181.42,181.58,177.55,18151903,medium
AAPL,1,2018,2,170.16,172.26,172.30,169.26,25555934,medium
MSFT,1,2018,2,86.13,85.95,86.31,85.50,22483797,medium


In [35]:
tech.loc[(slice(None),'FB'),:]

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2014-01-02,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
2014-01-03,FB,1,2014,3,55.00,54.56,55.65,54.53,38287706,medium
2014-01-06,FB,1,2014,6,54.39,57.20,57.26,54.05,68974359,high
2014-01-07,FB,1,2014,7,57.67,57.92,58.55,57.22,77329009,high
2014-01-08,FB,1,2014,8,57.59,58.23,58.41,57.23,56800776,high
...,...,...,...,...,...,...,...,...,...,...
2019-08-19,FB,8,2019,19,186.01,186.17,187.50,184.85,9699661,low
2019-08-20,FB,8,2019,20,185.45,183.81,186.00,182.39,10087592,low
2019-08-21,FB,8,2019,21,185.00,183.55,185.90,183.14,8409548,low


In [36]:
tech.xs('FB',level=1, drop_level=False)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2014-01-02,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
2014-01-03,FB,1,2014,3,55.00,54.56,55.65,54.53,38287706,medium
2014-01-06,FB,1,2014,6,54.39,57.20,57.26,54.05,68974359,high
2014-01-07,FB,1,2014,7,57.67,57.92,58.55,57.22,77329009,high
2014-01-08,FB,1,2014,8,57.59,58.23,58.41,57.23,56800776,high
...,...,...,...,...,...,...,...,...,...,...
2019-08-19,FB,8,2019,19,186.01,186.17,187.50,184.85,9699661,low
2019-08-20,FB,8,2019,20,185.45,183.81,186.00,182.39,10087592,low
2019-08-21,FB,8,2019,21,185.00,183.55,185.90,183.14,8409548,low


In [37]:
# selecting from multiple levels at once

In [38]:
tech.xs(('2018-01-02','FB'), level=(0,1), drop_level=False, axis=0)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2018-01-02,FB,1,2018,2,177.68,181.42,181.58,177.55,18151903,medium


In [39]:
# challenge
# 1. select all stocks from july13 2015 to august 17 2016 and save it to tech_df2
tech_df2 = tech.loc[slice('2015-07-13','2016-08-17'), :]

In [40]:
tech_df2

month  year  day    open   close    high     low    volume  \
date       name                                                                
2015-07-13 MSFT       7  2015   13   44.98   45.54   45.62   44.95  28178329   
           AMZN       7  2015   13  448.29  455.57  457.87  447.54   3956802   
           FB         7  2015   13   88.66   90.10   90.22   88.42  29976670   
           GOOGL      7  2015   13  559.51  571.73  572.85  558.70   2089641   
           AAPL       7  2015   13  125.03  125.66  125.76  124.32  41440538   
...                 ...   ...  ...     ...     ...     ...     ...       ...   
2016-08-17 GOOGL      8  2016   17  800.00  805.42  805.63  796.30   1066070   
           AMZN       8  2016   17  764.41  764.63  765.22  759.20   1891116   
           MSFT       8  2016   17   57.54   57.56   57.68   57.23  18856423   
           FB         8  2016   17  123.66  124.37  124.38  122.85  13794179   
           AAPL       8  2016   17  109.10  109.22  109.37  108.34  25355976   

                 volume_type  
date       name               
2015-07-13 MSFT       medium  
           AMZN       medium  
           FB         medium  
           GOOGL      medium  
           AAPL       medium  
...                      ...  
2016-08-17 GOOGL      medium  
           AMZN          low  
           MSFT       medium  
           FB            low  
           AAPL       medium  

[1395 rows x 9 columns]

In [41]:
# 2. select 10 days at random from tech_df2, but it should only contain AAPL
tech_df2.loc[(slice(None),'AAPL'), :].sample(10)
tech_df2.xs('AAPL', level=1, drop_level=False).sample(10)
tech_df2.loc[pd.IndexSlice[:,'AAPL'],:].sample(10)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2015-08-05,AAPL,8,2015,5,112.95,115.40,117.44,112.10,99312613,high
2015-11-09,AAPL,11,2015,9,120.96,120.57,121.81,120.05,33871405,medium
2016-01-28,AAPL,1,2016,28,93.79,94.09,94.52,92.39,55678825,medium
2016-04-13,AAPL,4,2016,13,110.80,112.04,112.34,110.80,33257316,medium
2016-02-09,AAPL,2,2016,9,94.29,94.99,95.94,93.93,44331195,medium
2016-06-09,AAPL,6,2016,9,98.50,99.65,99.99,98.46,26601354,medium
2015-11-18,AAPL,11,2015,18,115.76,117.29,117.49,115.50,46674697,medium
2015-12-16,AAPL,12,2015,16,111.07,111.34,111.99,108.80,56238467,medium
2015-09-10,AAPL,9,2015,10,110.27,112.57,113.28,109.90,62892831,medium


In [42]:
# 3. select all high and low of AAPL & GOOGL for all the dates in tech_df2

tech_df2.loc[pd.IndexSlice[:,['AAPL','GOOGL']], 'high':'low']
tech_df2.loc[(slice(None),['AAPL','GOOGL']),'high':'low' ]

,,high,low
date,name,,
2015-07-13,AAPL,125.76,124.32
2015-07-14,AAPL,126.37,125.04
2015-07-15,AAPL,127.15,125.58
2015-07-16,AAPL,128.57,127.35
2015-07-17,AAPL,129.62,128.31
...,...,...,...
2016-08-11,GOOGL,813.88,806.00
2016-08-12,GOOGL,807.19,803.64
2016-08-15,GOOGL,811.36,804.03


### Anatomy of MultiIndex

In [43]:
type(tech.index)

pandas.core.indexes.multi.MultiIndex

In [44]:
tech.ndim

2

In [45]:
tech.index

MultiIndex([('2014-01-02',    'FB'),
            ('2014-01-02',  'AAPL'),
            ('2014-01-02', 'GOOGL'),
            ('2014-01-02',  'MSFT'),
            ('2014-01-02',  'AMZN'),
            ('2014-01-03',    'FB'),
            ('2014-01-03', 'GOOGL'),
            ('2014-01-03',  'MSFT'),
            ('2014-01-03',  'AAPL'),
            ('2014-01-03',  'AMZN'),
            ...
            ('2019-08-22',  'MSFT'),
            ('2019-08-22',    'FB'),
            ('2019-08-22',  'AMZN'),
            ('2019-08-22',  'AAPL'),
            ('2019-08-22', 'GOOGL'),
            ('2019-08-23',  'MSFT'),
            ('2019-08-23',  'AAPL'),
            ('2019-08-23', 'GOOGL'),
            ('2019-08-23',  'AMZN'),
            ('2019-08-23',    'FB')],
           names=['date', 'name'], length=7105)

In [46]:
tech.index.names

FrozenList(['date', 'name'])

In [47]:
tech.index.nlevels, tech.index.levels

(2,
 FrozenList([['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-17', '2014-01-21', '2014-01-22', '2014-01-23', '2014-01-24', '2014-01-27', '2014-01-28', '2014-01-29', '2014-01-30', '2014-01-31', '2014-02-03', '2014-02-04', '2014-02-05', '2014-02-06', '2014-02-07', '2014-02-10', '2014-02-11', '2014-02-12', '2014-02-13', '2014-02-14', '2014-02-18', '2014-02-19', '2014-02-20', '2014-02-21', '2014-02-24', '2014-02-25', '2014-02-26', '2014-02-27', '2014-02-28', '2014-03-03', '2014-03-04', '2014-03-05', '2014-03-06', '2014-03-07', '2014-03-10', '2014-03-11', '2014-03-12', '2014-03-13', '2014-03-14', '2014-03-17', '2014-03-18', '2014-03-19', '2014-03-20', '2014-03-21', '2014-03-24', '2014-03-25', '2014-03-26', '2014-03-27', '2014-03-28', '2014-03-31', '2014-04-01', '2014-04-02', '2014-04-03', '2014-04-04', '2014-04-07', '2014-04-08', '2014-04-09', '2014-04-10', '2014-04-11', '2

In [48]:
tech.index.levels[0]

Index(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', '2014-01-08',
       '2014-01-09', '2014-01-10', '2014-01-13', '2014-01-14', '2014-01-15',
       ...
       '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
       '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22', '2019-08-23'],
      dtype='object', name='date', length=1421)

In [49]:
tech.index.levels[1]

Index(['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT'], dtype='object', name='name')

In [50]:
tech.index.levshape

(1421, 5)

In [51]:
tech.index.values

array([('2014-01-02', 'FB'), ('2014-01-02', 'AAPL'),
       ('2014-01-02', 'GOOGL'), ..., ('2019-08-23', 'GOOGL'),
       ('2019-08-23', 'AMZN'), ('2019-08-23', 'FB')], dtype=object)

### Adding another level

In [52]:
tech.set_index('volume_type', append=True, inplace=True)

In [53]:
tech.head()

month  year  day    open   close    high  \
date       name  volume_type                                             
2014-01-02 FB    medium           1  2014    2   54.86   54.71   55.22   
           AAPL  low              1  2014    2   79.38   79.02   79.58   
           GOOGL medium           1  2014    2  557.73  556.56  558.88   
           MSFT  medium           1  2014    2   37.35   37.16   37.40   
           AMZN  medium           1  2014    2  398.80  397.97  399.36   

                                 low    volume  
date       name  volume_type                    
2014-01-02 FB    medium        54.19  43257622  
           AAPL  low           78.86   8398851  
           GOOGL medium       554.13   1822719  
           MSFT  medium        37.10  30643745  
           AMZN  medium       394.02   2140246

In [54]:
tech.index.nlevels, tech.index.levels[2]

(3, Index(['high', 'low', 'medium'], dtype='object', name='volume_type'))

In [55]:
tech.index.levshape

(1421, 5, 3)

In [56]:
tech.index.values

array([('2014-01-02', 'FB', 'medium'), ('2014-01-02', 'AAPL', 'low'),
       ('2014-01-02', 'GOOGL', 'medium'), ...,
       ('2019-08-23', 'GOOGL', 'medium'),
       ('2019-08-23', 'AMZN', 'medium'), ('2019-08-23', 'FB', 'medium')],
      dtype=object)

In [57]:
# high volume trading days of jan 2019
tech.loc[(slice('2019-01-01','2019-01-31'), slice(None), 'high'),:]

month  year  day     open    close     high  \
date       name volume_type                                                
2019-01-03 AAPL high             1  2019    3   143.98   142.19   145.72   
2019-01-04 AMZN high             1  2019    4  1530.00  1575.39  1594.00   
2019-01-08 AMZN high             1  2019    8  1664.69  1656.58  1676.61   
2019-01-31 AMZN high             1  2019   31  1692.85  1718.73  1736.41   
           FB   high             1  2019   31   165.60   166.69   171.68   

                                 low    volume  
date       name volume_type                     
2019-01-03 AAPL high          142.00  91312195  
2019-01-04 AMZN high         1518.31   9182575  
2019-01-08 AMZN high         1616.61   8881428  
2019-01-31 AMZN high         1679.08  10910338  
           FB   high          165.00  77233602

In [58]:
# all high volume trading days
tech.xs('high', level=2, drop_level=False)

month  year  day     open    close     high  \
date       name  volume_type                                                
2014-01-06 FB    high             1  2014    6    54.39    57.20    57.26   
2014-01-07 FB    high             1  2014    7    57.67    57.92    58.55   
2014-01-08 FB    high             1  2014    8    57.59    58.23    58.41   
2014-01-09 FB    high             1  2014    9    58.66    57.22    58.96   
2014-01-13 FB    high             1  2014   13    57.89    55.91    58.25   
...                             ...   ...  ...      ...      ...      ...   
2019-04-30 GOOGL high             4  2019   30  1190.63  1198.96  1200.98   
2019-06-03 FB    high             6  2019    3   175.00   164.15   175.05   
           AMZN  high             6  2019    3  1760.01  1692.69  1766.29   
           GOOGL high             6  2019    3  1066.93  1038.74  1067.00   
2019-07-26 GOOGL high             7  2019   26  1228.00  1245.22  1268.39   

                                  low    volume  
date       name  volume_type                     
2014-01-06 FB    high           54.05  68974359  
2014-01-07 FB    high           57.22  77329009  
2014-01-08 FB    high           57.23  56800776  
2014-01-09 FB    high           56.65  92349222  
2014-01-13 FB    high           55.38  63106519  
...                               ...       ...  
2019-04-30 GOOGL high         1183.00   6658855  
2019-06-03 FB    high          161.01  56059609  
           AMZN  high         1672.00   9098708  
           GOOGL high         1027.03   4844480  
2019-07-26 GOOGL high         1228.00   6060795  

[375 rows x 8 columns]

In [59]:
tech.xs(key=('FB','high'), level=(1,2), drop_level=False)

,,,month,year,day,open,close,high,low,volume
date,name,volume_type,,,,,,,,
2014-01-06,FB,high,1,2014,6,54.39,57.20,57.26,54.05,68974359
2014-01-07,FB,high,1,2014,7,57.67,57.92,58.55,57.22,77329009
2014-01-08,FB,high,1,2014,8,57.59,58.23,58.41,57.23,56800776
2014-01-09,FB,high,1,2014,9,58.66,57.22,58.96,56.65,92349222
2014-01-13,FB,high,1,2014,13,57.89,55.91,58.25,55.38,63106519
...,...,...,...,...,...,...,...,...,...,...
2018-10-31,FB,high,10,2018,31,155.00,151.79,156.40,148.96,60101251
2018-12-19,FB,high,12,2018,19,141.21,133.24,144.91,132.50,57404894
2018-12-21,FB,high,12,2018,21,133.39,124.95,134.90,123.42,56901491


### shuffling levels

In [60]:
tech.swaplevel(i = 2,j = 1)

month  year  day     open    close     high  \
date       volume_type name                                                 
2014-01-02 medium      FB         1  2014    2    54.86    54.71    55.22   
           low         AAPL       1  2014    2    79.38    79.02    79.58   
           medium      GOOGL      1  2014    2   557.73   556.56   558.88   
                       MSFT       1  2014    2    37.35    37.16    37.40   
                       AMZN       1  2014    2   398.80   397.97   399.36   
...                             ...   ...  ...      ...      ...      ...   
2019-08-23 medium      MSFT       8  2019   23   137.19   133.39   138.35   
                       AAPL       8  2019   23   209.43   202.64   212.05   
                       GOOGL      8  2019   23  1185.17  1153.58  1195.67   
                       AMZN       8  2019   23  1793.03  1749.62  1804.90   
                       FB         8  2019   23   180.84   177.75   183.13   

                                  low    volume  
date       volume_type name                      
2014-01-02 medium      FB       54.19  43257622  
           low         AAPL     78.86   8398851  
           medium      GOOGL   554.13   1822719  
                       MSFT     37.10  30643745  
                       AMZN    394.02   2140246  
...                               ...       ...  
2019-08-23 medium      MSFT    132.80  38515386  
                       AAPL    201.00  46882843  
                       GOOGL  1150.00   1813141  
                       AMZN   1745.23   5277898  
                       FB      176.66  17331221  

[7105 rows x 8 columns]

In [61]:
tech = tech.swaplevel('volume_type', 'name')

In [62]:
tech.head()

month  year  day    open   close    high  \
date       volume_type name                                              
2014-01-02 medium      FB         1  2014    2   54.86   54.71   55.22   
           low         AAPL       1  2014    2   79.38   79.02   79.58   
           medium      GOOGL      1  2014    2  557.73  556.56  558.88   
                       MSFT       1  2014    2   37.35   37.16   37.40   
                       AMZN       1  2014    2  398.80  397.97  399.36   

                                 low    volume  
date       volume_type name                     
2014-01-02 medium      FB      54.19  43257622  
           low         AAPL    78.86   8398851  
           medium      GOOGL  554.13   1822719  
                       MSFT    37.10  30643745  
                       AMZN   394.02   2140246

In [63]:
# reorder_level()
tech.reorder_levels([2,0,1])

,,,month,year,day,open,close,high,low,volume
name,date,volume_type,,,,,,,,
FB,2014-01-02,medium,1,2014,2,54.86,54.71,55.22,54.19,43257622
AAPL,2014-01-02,low,1,2014,2,79.38,79.02,79.58,78.86,8398851
GOOGL,2014-01-02,medium,1,2014,2,557.73,556.56,558.88,554.13,1822719
MSFT,2014-01-02,medium,1,2014,2,37.35,37.16,37.40,37.10,30643745
AMZN,2014-01-02,medium,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...
MSFT,2019-08-23,medium,8,2019,23,137.19,133.39,138.35,132.80,38515386
AAPL,2019-08-23,medium,8,2019,23,209.43,202.64,212.05,201.00,46882843
GOOGL,2019-08-23,medium,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


In [64]:
tech.index.reorder_levels([2,0,1])

MultiIndex([(   'FB', '2014-01-02', 'medium'),
            ( 'AAPL', '2014-01-02',    'low'),
            ('GOOGL', '2014-01-02', 'medium'),
            ( 'MSFT', '2014-01-02', 'medium'),
            ( 'AMZN', '2014-01-02', 'medium'),
            (   'FB', '2014-01-03', 'medium'),
            ('GOOGL', '2014-01-03', 'medium'),
            ( 'MSFT', '2014-01-03', 'medium'),
            ( 'AAPL', '2014-01-03',    'low'),
            ( 'AMZN', '2014-01-03', 'medium'),
            ...
            ( 'MSFT', '2019-08-22', 'medium'),
            (   'FB', '2019-08-22',    'low'),
            ( 'AMZN', '2019-08-22', 'medium'),
            ( 'AAPL', '2019-08-22', 'medium'),
            ('GOOGL', '2019-08-22',    'low'),
            ( 'MSFT', '2019-08-23', 'medium'),
            ( 'AAPL', '2019-08-23', 'medium'),
            ('GOOGL', '2019-08-23', 'medium'),
            ( 'AMZN', '2019-08-23', 'medium'),
            (   'FB', '2019-08-23', 'medium')],
           names=['name', 'date', 'volume_t

### Removing MultiIndex levels

In [65]:
tech.droplevel(1)

month  year  day     open    close     high      low  \
date       name                                                          
2014-01-02 FB         1  2014    2    54.86    54.71    55.22    54.19   
           AAPL       1  2014    2    79.38    79.02    79.58    78.86   
           GOOGL      1  2014    2   557.73   556.56   558.88   554.13   
           MSFT       1  2014    2    37.35    37.16    37.40    37.10   
           AMZN       1  2014    2   398.80   397.97   399.36   394.02   
...                 ...   ...  ...      ...      ...      ...      ...   
2019-08-23 MSFT       8  2019   23   137.19   133.39   138.35   132.80   
           AAPL       8  2019   23   209.43   202.64   212.05   201.00   
           GOOGL      8  2019   23  1185.17  1153.58  1195.67  1150.00   
           AMZN       8  2019   23  1793.03  1749.62  1804.90  1745.23   
           FB         8  2019   23   180.84   177.75   183.13   176.66   

                    volume  
date       name             
2014-01-02 FB     43257622  
           AAPL    8398851  
           GOOGL   1822719  
           MSFT   30643745  
           AMZN    2140246  
...                    ...  
2019-08-23 MSFT   38515386  
           AAPL   46882843  
           GOOGL   1813141  
           AMZN    5277898  
           FB     17331221  

[7105 rows x 8 columns]

In [66]:
tech.droplevel([0,1])

,month,year,day,open,close,high,low,volume
name,,,,,,,,
FB,1,2014,2,54.86,54.71,55.22,54.19,43257622
AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851
GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719
MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745
AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...
MSFT,8,2019,23,137.19,133.39,138.35,132.80,38515386
AAPL,8,2019,23,209.43,202.64,212.05,201.00,46882843
GOOGL,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


In [67]:
tech.reset_index(level=1, drop=False)

volume_type  month  year  day     open    close     high  \
date       name                                                             
2014-01-02 FB         medium      1  2014    2    54.86    54.71    55.22   
           AAPL          low      1  2014    2    79.38    79.02    79.58   
           GOOGL      medium      1  2014    2   557.73   556.56   558.88   
           MSFT       medium      1  2014    2    37.35    37.16    37.40   
           AMZN       medium      1  2014    2   398.80   397.97   399.36   
...                      ...    ...   ...  ...      ...      ...      ...   
2019-08-23 MSFT       medium      8  2019   23   137.19   133.39   138.35   
           AAPL       medium      8  2019   23   209.43   202.64   212.05   
           GOOGL      medium      8  2019   23  1185.17  1153.58  1195.67   
           AMZN       medium      8  2019   23  1793.03  1749.62  1804.90   
           FB         medium      8  2019   23   180.84   177.75   183.13   

                      low    volume  
date       name                      
2014-01-02 FB       54.19  43257622  
           AAPL     78.86   8398851  
           GOOGL   554.13   1822719  
           MSFT     37.10  30643745  
           AMZN    394.02   2140246  
...                   ...       ...  
2019-08-23 MSFT    132.80  38515386  
           AAPL    201.00  46882843  
           GOOGL  1150.00   1813141  
           AMZN   1745.23   5277898  
           FB      176.66  17331221  

[7105 rows x 9 columns]

In [68]:
tech.reset_index(level=[0,1])

,date,volume_type,month,year,day,open,close,high,low,volume
name,,,,,,,,,,
FB,2014-01-02,medium,1,2014,2,54.86,54.71,55.22,54.19,43257622
AAPL,2014-01-02,low,1,2014,2,79.38,79.02,79.58,78.86,8398851
GOOGL,2014-01-02,medium,1,2014,2,557.73,556.56,558.88,554.13,1822719
MSFT,2014-01-02,medium,1,2014,2,37.35,37.16,37.40,37.10,30643745
AMZN,2014-01-02,medium,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...
MSFT,2019-08-23,medium,8,2019,23,137.19,133.39,138.35,132.80,38515386
AAPL,2019-08-23,medium,8,2019,23,209.43,202.64,212.05,201.00,46882843
GOOGL,2019-08-23,medium,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


In [69]:
tech.reset_index()  # total reset

,date,volume_type,name,month,year,day,open,close,high,low,volume
0,2014-01-02,medium,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622
1,2014-01-02,low,AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851
2,2014-01-02,medium,GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719
3,2014-01-02,medium,MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745
4,2014-01-02,medium,AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...,...
7100,2019-08-23,medium,MSFT,8,2019,23,137.19,133.39,138.35,132.80,38515386
7101,2019-08-23,medium,AAPL,8,2019,23,209.43,202.64,212.05,201.00,46882843
7102,2019-08-23,medium,GOOGL,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141
7103,2019-08-23,medium,AMZN,8,2019,23,1793.03,1749.62,1804.90,1745.23,5277898


### MultiIndex sort_index()

In [70]:
tech.loc[(slice('2019-01-01','2019-01-31'), slice(None), slice('AAPL', 'FB'))]

UnsortedIndexError: 'MultiIndex slicing requires the index to be lexsorted: slicing on levels [0, 2], lexsort depth 1'

In [71]:
tech.head(10)

month  year  day    open   close    high  \
date       volume_type name                                              
2014-01-02 medium      FB         1  2014    2   54.86   54.71   55.22   
           low         AAPL       1  2014    2   79.38   79.02   79.58   
           medium      GOOGL      1  2014    2  557.73  556.56  558.88   
                       MSFT       1  2014    2   37.35   37.16   37.40   
                       AMZN       1  2014    2  398.80  397.97  399.36   
2014-01-03 medium      FB         1  2014    3   55.00   54.56   55.65   
                       GOOGL      1  2014    3  557.50  552.50  558.47   
                       MSFT       1  2014    3   37.20   36.91   37.22   
           low         AAPL       1  2014    3   79.00   77.28   79.10   
           medium      AMZN       1  2014    3  398.29  396.44  402.71   

                                 low    volume  
date       volume_type name                     
2014-01-02 medium      FB      54.19  43257622  
           low         AAPL    78.86   8398851  
           medium      GOOGL  554.13   1822719  
                       MSFT    37.10  30643745  
                       AMZN   394.02   2140246  
2014-01-03 medium      FB      54.53  38287706  
                       GOOGL  552.47   1669229  
                       MSFT    36.60  31134795  
           low         AAPL    77.20  14043410  
           medium      AMZN   396.22   2213512

In [72]:
tech.sort_index(inplace=True)

In [73]:
tech.loc[(slice('2019-01-01','2019-01-31'), slice(None), slice('AAPL', 'FB'))]

month  year  day     open    close     high  \
date       volume_type name                                                
2019-01-02 medium      AAPL      1  2019    2   154.89   157.92   158.85   
                       AMZN      1  2019    2  1465.20  1539.13  1553.36   
                       FB        1  2019    2   128.99   135.68   137.51   
2019-01-03 high        AAPL      1  2019    3   143.98   142.19   145.72   
           medium      AMZN      1  2019    3  1520.01  1500.28  1538.00   
...                            ...   ...  ...      ...      ...      ...   
2019-01-30 medium      AMZN      1  2019   30  1623.00  1670.43  1676.95   
                       FB        1  2019   30   146.22   150.42   150.95   
2019-01-31 high        AMZN      1  2019   31  1692.85  1718.73  1736.41   
                       FB        1  2019   31   165.60   166.69   171.68   
           medium      AAPL      1  2019   31   166.11   166.44   169.00   

                                 low    volume  
date       volume_type name                     
2019-01-02 medium      AAPL   154.23  37039737  
                       AMZN  1460.93   7983103  
                       FB     128.56  28146193  
2019-01-03 high        AAPL   142.00  91312195  
           medium      AMZN  1497.11   6975572  
...                              ...       ...  
2019-01-30 medium      AMZN  1619.68   5783822  
                       FB     145.70  44613240  
2019-01-31 high        AMZN  1679.08  10910338  
                       FB     165.00  77233602  
           medium      AAPL   164.56  40739649  

[63 rows x 8 columns]

In [74]:
# Always consider sorting index
# it improves retrieval performance
# enables slicing index
# overall a good practice when working with tabular data sets(pandas, excel, sql)

In [75]:
tech.sort_index(level=(0,2), ascending=[False,True])

month  year  day     open    close     high  \
date       volume_type name                                                 
2019-08-23 medium      AAPL       8  2019   23   209.43   202.64   212.05   
                       AMZN       8  2019   23  1793.03  1749.62  1804.90   
                       FB         8  2019   23   180.84   177.75   183.13   
                       GOOGL      8  2019   23  1185.17  1153.58  1195.67   
                       MSFT       8  2019   23   137.19   133.39   138.35   
...                             ...   ...  ...      ...      ...      ...   
2014-01-02 low         AAPL       1  2014    2    79.38    79.02    79.58   
           medium      AMZN       1  2014    2   398.80   397.97   399.36   
                       FB         1  2014    2    54.86    54.71    55.22   
                       GOOGL      1  2014    2   557.73   556.56   558.88   
                       MSFT       1  2014    2    37.35    37.16    37.40   

                                  low    volume  
date       volume_type name                      
2019-08-23 medium      AAPL    201.00  46882843  
                       AMZN   1745.23   5277898  
                       FB      176.66  17331221  
                       GOOGL  1150.00   1813141  
                       MSFT    132.80  38515386  
...                               ...       ...  
2014-01-02 low         AAPL     78.86   8398851  
           medium      AMZN    394.02   2140246  
                       FB       54.19  43257622  
                       GOOGL   554.13   1822719  
                       MSFT     37.10  30643745  

[7105 rows x 8 columns]

### More MultiIndex methods

In [76]:
tech.index.is_lexsorted()

C:\Users\sonig\AppData\Local\Temp\ipykernel_23948\1988919264.py:1: FutureWarning: MultiIndex.is_lexsorted is deprecated as a public function, users should use MultiIndex.is_monotonic_increasing instead.
  tech.index.is_lexsorted()


True

In [77]:
tech.index.is_monotonic_increasing

True

In [78]:
tech.index.sortlevel(0, ascending=False, sort_remaining=True)  # works only on index, values is not sorted.

(MultiIndex([('2019-08-23', 'medium',  'MSFT'),
             ('2019-08-23', 'medium', 'GOOGL'),
             ('2019-08-23', 'medium',    'FB'),
             ('2019-08-23', 'medium',  'AMZN'),
             ('2019-08-23', 'medium',  'AAPL'),
             ('2019-08-22', 'medium',  'MSFT'),
             ('2019-08-22', 'medium',  'AMZN'),
             ('2019-08-22', 'medium',  'AAPL'),
             ('2019-08-22',    'low', 'GOOGL'),
             ('2019-08-22',    'low',    'FB'),
             ...
             ('2014-01-03', 'medium',  'MSFT'),
             ('2014-01-03', 'medium', 'GOOGL'),
             ('2014-01-03', 'medium',    'FB'),
             ('2014-01-03', 'medium',  'AMZN'),
             ('2014-01-03',    'low',  'AAPL'),
             ('2014-01-02', 'medium',  'MSFT'),
             ('2014-01-02', 'medium', 'GOOGL'),
             ('2014-01-02', 'medium',    'FB'),
             ('2014-01-02', 'medium',  'AMZN'),
             ('2014-01-02',    'low',  'AAPL')],
            names=['da

In [79]:
tech.head()

month  year  day    open   close    high  \
date       volume_type name                                              
2014-01-02 low         AAPL       1  2014    2   79.38   79.02   79.58   
           medium      AMZN       1  2014    2  398.80  397.97  399.36   
                       FB         1  2014    2   54.86   54.71   55.22   
                       GOOGL      1  2014    2  557.73  556.56  558.88   
                       MSFT       1  2014    2   37.35   37.16   37.40   

                                 low    volume  
date       volume_type name                     
2014-01-02 low         AAPL    78.86   8398851  
           medium      AMZN   394.02   2140246  
                       FB      54.19  43257622  
                       GOOGL  554.13   1822719  
                       MSFT    37.10  30643745

In [80]:
# set_names()
tech.index.set_names(['Trading Date', 'Volume Category', 'Ticker'], inplace=True)

In [81]:
tech.head()

month  year  day    open   close    high  \
Trading Date Volume Category Ticker                                             
2014-01-02   low             AAPL        1  2014    2   79.38   79.02   79.58   
             medium          AMZN        1  2014    2  398.80  397.97  399.36   
                             FB          1  2014    2   54.86   54.71   55.22   
                             GOOGL       1  2014    2  557.73  556.56  558.88   
                             MSFT        1  2014    2   37.35   37.16   37.40   

                                        low    volume  
Trading Date Volume Category Ticker                    
2014-01-02   low             AAPL     78.86   8398851  
             medium          AMZN    394.02   2140246  
                             FB       54.19  43257622  
                             GOOGL   554.13   1822719  
                             MSFT     37.10  30643745

In [82]:
tech.index

MultiIndex([('2014-01-02',    'low',  'AAPL'),
            ('2014-01-02', 'medium',  'AMZN'),
            ('2014-01-02', 'medium',    'FB'),
            ('2014-01-02', 'medium', 'GOOGL'),
            ('2014-01-02', 'medium',  'MSFT'),
            ('2014-01-03',    'low',  'AAPL'),
            ('2014-01-03', 'medium',  'AMZN'),
            ('2014-01-03', 'medium',    'FB'),
            ('2014-01-03', 'medium', 'GOOGL'),
            ('2014-01-03', 'medium',  'MSFT'),
            ...
            ('2019-08-22',    'low',    'FB'),
            ('2019-08-22',    'low', 'GOOGL'),
            ('2019-08-22', 'medium',  'AAPL'),
            ('2019-08-22', 'medium',  'AMZN'),
            ('2019-08-22', 'medium',  'MSFT'),
            ('2019-08-23', 'medium',  'AAPL'),
            ('2019-08-23', 'medium',  'AMZN'),
            ('2019-08-23', 'medium',    'FB'),
            ('2019-08-23', 'medium', 'GOOGL'),
            ('2019-08-23', 'medium',  'MSFT')],
           names=['Trading Date', 'Volume C

In [86]:
tech.index.to_flat_index()

Index([    ('2014-01-02', 'low', 'AAPL'),  ('2014-01-02', 'medium', 'AMZN'),
          ('2014-01-02', 'medium', 'FB'), ('2014-01-02', 'medium', 'GOOGL'),
        ('2014-01-02', 'medium', 'MSFT'),     ('2014-01-03', 'low', 'AAPL'),
        ('2014-01-03', 'medium', 'AMZN'),    ('2014-01-03', 'medium', 'FB'),
       ('2014-01-03', 'medium', 'GOOGL'),  ('2014-01-03', 'medium', 'MSFT'),
       ...
             ('2019-08-22', 'low', 'FB'),    ('2019-08-22', 'low', 'GOOGL'),
        ('2019-08-22', 'medium', 'AAPL'),  ('2019-08-22', 'medium', 'AMZN'),
        ('2019-08-22', 'medium', 'MSFT'),  ('2019-08-23', 'medium', 'AAPL'),
        ('2019-08-23', 'medium', 'AMZN'),    ('2019-08-23', 'medium', 'FB'),
       ('2019-08-23', 'medium', 'GOOGL'),  ('2019-08-23', 'medium', 'MSFT')],
      dtype='object', length=7105)

In [87]:
tech.close

Trading Date  Volume Category  Ticker
2014-01-02    low              AAPL        79.02
              medium           AMZN       397.97
                               FB          54.71
                               GOOGL      556.56
                               MSFT        37.16
                                          ...   
2019-08-23    medium           AAPL       202.64
                               AMZN      1749.62
                               FB         177.75
                               GOOGL     1153.58
                               MSFT       133.39
Name: close, Length: 7105, dtype: float64

### stack()

In [88]:
stacked = tech.stack()

In [89]:
stacked.head(10)  # it is a series

Trading Date  Volume Category  Ticker        
2014-01-02    low              AAPL    month           1.00
                                       year         2014.00
                                       day             2.00
                                       open           79.38
                                       close          79.02
                                       high           79.58
                                       low            78.86
                                       volume    8398851.00
              medium           AMZN    month           1.00
                                       year         2014.00
dtype: float64

In [90]:
stacked.index.nlevels, type(stacked)

(4, pandas.core.series.Series)

In [93]:
names = stacked.index.names

In [97]:
stacked.index.set_names([*names[:-1], 'Previously a column axis'], inplace=True)

In [98]:
stacked

Trading Date  Volume Category  Ticker  Previously a column axis
2014-01-02    low              AAPL    month                              1.00
                                       year                            2014.00
                                       day                                2.00
                                       open                              79.38
                                       close                             79.02
                                                                      ...     
2019-08-23    medium           MSFT    open                             137.19
                                       close                            133.39
                                       high                             138.35
                                       low                              132.80
                                       volume                      38515386.00
Length: 56840, dtype: float64

### unstack()

In [99]:
stacked.unstack()

Previously a column axis             month    year   day     open    close  \
Trading Date Volume Category Ticker                                          
2014-01-02   low             AAPL      1.0  2014.0   2.0    79.38    79.02   
             medium          AMZN      1.0  2014.0   2.0   398.80   397.97   
                             FB        1.0  2014.0   2.0    54.86    54.71   
                             GOOGL     1.0  2014.0   2.0   557.73   556.56   
                             MSFT      1.0  2014.0   2.0    37.35    37.16   
...                                    ...     ...   ...      ...      ...   
2019-08-23   medium          AAPL      8.0  2019.0  23.0   209.43   202.64   
                             AMZN      8.0  2019.0  23.0  1793.03  1749.62   
                             FB        8.0  2019.0  23.0   180.84   177.75   
                             GOOGL     8.0  2019.0  23.0  1185.17  1153.58   
                             MSFT      8.0  2019.0  23.0   137.19   133.39   

Previously a column axis                high      low      volume  
Trading Date Volume Category Ticker                                
2014-01-02   low             AAPL      79.58    78.86   8398851.0  
             medium          AMZN     399.36   394.02   2140246.0  
                             FB        55.22    54.19  43257622.0  
                             GOOGL    558.88   554.13   1822719.0  
                             MSFT      37.40    37.10  30643745.0  
...                                      ...      ...         ...  
2019-08-23   medium          AAPL     212.05   201.00  46882843.0  
                             AMZN    1804.90  1745.23   5277898.0  
                             FB       183.13   176.66  17331221.0  
                             GOOGL   1195.67  1150.00   1813141.0  
                             MSFT     138.35   132.80  38515386.0  

[7105 rows x 8 columns]

In [100]:
stacked.unstack().unstack()

Previously a column axis     month                         year          \
Ticker                        AAPL AMZN   FB GOOGL MSFT    AAPL    AMZN   
Trading Date Volume Category                                              
2014-01-02   low               1.0  NaN  NaN   NaN  NaN  2014.0     NaN   
             medium            NaN  1.0  1.0   1.0  1.0     NaN  2014.0   
2014-01-03   low               1.0  NaN  NaN   NaN  NaN  2014.0     NaN   
             medium            NaN  1.0  1.0   1.0  1.0     NaN  2014.0   
2014-01-06   high              NaN  NaN  1.0   NaN  NaN     NaN     NaN   
...                            ...  ...  ...   ...  ...     ...     ...   
2019-08-21   low               NaN  8.0  8.0   8.0  8.0     NaN  2019.0   
             medium            8.0  NaN  NaN   NaN  NaN  2019.0     NaN   
2019-08-22   low               NaN  NaN  8.0   8.0  NaN     NaN     NaN   
             medium            8.0  8.0  NaN   NaN  8.0  2019.0  2019.0   
2019-08-23   medium            8.0  8.0  8.0   8.0  8.0  2019.0  2019.0   

Previously a column axis                              ...     low           \
Ticker                            FB   GOOGL    MSFT  ...    AAPL     AMZN   
Trading Date Volume Category                          ...                    
2014-01-02   low                 NaN     NaN     NaN  ...   78.86      NaN   
             medium           2014.0  2014.0  2014.0  ...     NaN   394.02   
2014-01-03   low                 NaN     NaN     NaN  ...   77.20      NaN   
             medium           2014.0  2014.0  2014.0  ...     NaN   396.22   
2014-01-06   high             2014.0     NaN     NaN  ...     NaN      NaN   
...                              ...     ...     ...  ...     ...      ...   
2019-08-21   low              2019.0  2019.0  2019.0  ...     NaN  1815.00   
             medium              NaN     NaN     NaN  ...  211.60      NaN   
2019-08-22   low              2019.0  2019.0     NaN  ...     NaN      NaN   
             medium              NaN     NaN  2019.0  ...  210.75  1800.10   
2019-08-23   medium           2019.0  2019.0  2019.0  ...  201.00  1745.23   

Previously a column axis                                   volume             \
Ticker                            FB    GOOGL    MSFT        AAPL       AMZN   
Trading Date Volume Category                                                   
2014-01-02   low                 NaN      NaN     NaN   8398851.0        NaN   
             medium            54.19   554.13   37.10         NaN  2140246.0   
2014-01-03   low                 NaN      NaN     NaN  14043410.0        NaN   
             medium            54.53   552.47   36.60         NaN  2213512.0   
2014-01-06   high              54.05      NaN     NaN         NaN        NaN   
...                              ...      ...     ...         ...        ...   
2019-08-21   low              183.14  1187.92  138.00         NaN  2039231.0   
             medium              NaN      NaN     NaN  21564747.0        NaN   
2019-08-22   low              179.91  1178.91     NaN         NaN        NaN   
             medium              NaN      NaN  136.29  22267819.0  2658388.0   
2019-08-23   medium           176.66  1150.00  132.80  46882843.0  5277898.0   

Previously a column axis                                         
Ticker                                FB      GOOGL        MSFT  
Trading Date Volume Category                                     
2014-01-02   low                     NaN        NaN         NaN  
             medium           43257622.0  1822719.0  30643745.0  
2014-01-03   low                     NaN        NaN         NaN  
             medium           38287706.0  1669229.0  31134795.0  
2014-01-06   high             68974359.0        NaN         NaN  
...                                  ...        ...         ...  
2019-08-21   low               8409548.0   708272.0  14982314.0  
             medium                  NaN        NaN         NaN  
2019-08-2

In [101]:
stacked.unstack().unstack(fill_value='-')

Previously a column axis     month                         year          \
Ticker                        AAPL AMZN   FB GOOGL MSFT    AAPL    AMZN   
Trading Date Volume Category                                              
2014-01-02   low               1.0    -    -     -    -  2014.0       -   
             medium              -  1.0  1.0   1.0  1.0       -  2014.0   
2014-01-03   low               1.0    -    -     -    -  2014.0       -   
             medium              -  1.0  1.0   1.0  1.0       -  2014.0   
2014-01-06   high                -    -  1.0     -    -       -       -   
...                            ...  ...  ...   ...  ...     ...     ...   
2019-08-21   low                 -  8.0  8.0   8.0  8.0       -  2019.0   
             medium            8.0    -    -     -    -  2019.0       -   
2019-08-22   low                 -    -  8.0   8.0    -       -       -   
             medium            8.0  8.0    -     -  8.0  2019.0  2019.0   
2019-08-23   medium            8.0  8.0  8.0   8.0  8.0  2019.0  2019.0   

Previously a column axis                              ...     low           \
Ticker                            FB   GOOGL    MSFT  ...    AAPL     AMZN   
Trading Date Volume Category                          ...                    
2014-01-02   low                   -       -       -  ...   78.86        -   
             medium           2014.0  2014.0  2014.0  ...       -   394.02   
2014-01-03   low                   -       -       -  ...    77.2        -   
             medium           2014.0  2014.0  2014.0  ...       -   396.22   
2014-01-06   high             2014.0       -       -  ...       -        -   
...                              ...     ...     ...  ...     ...      ...   
2019-08-21   low              2019.0  2019.0  2019.0  ...       -   1815.0   
             medium                -       -       -  ...   211.6        -   
2019-08-22   low              2019.0  2019.0       -  ...       -        -   
             medium                -       -  2019.0  ...  210.75   1800.1   
2019-08-23   medium           2019.0  2019.0  2019.0  ...   201.0  1745.23   

Previously a column axis                                   volume             \
Ticker                            FB    GOOGL    MSFT        AAPL       AMZN   
Trading Date Volume Category                                                   
2014-01-02   low                   -        -       -   8398851.0          -   
             medium            54.19   554.13    37.1           -  2140246.0   
2014-01-03   low                   -        -       -  14043410.0          -   
             medium            54.53   552.47    36.6           -  2213512.0   
2014-01-06   high              54.05        -       -           -          -   
...                              ...      ...     ...         ...        ...   
2019-08-21   low              183.14  1187.92   138.0           -  2039231.0   
             medium                -        -       -  21564747.0          -   
2019-08-22   low              179.91  1178.91       -           -          -   
             medium                -        -  136.29  22267819.0  2658388.0   
2019-08-23   medium           176.66   1150.0   132.8  46882843.0  5277898.0   

Previously a column axis                                         
Ticker                                FB      GOOGL        MSFT  
Trading Date Volume Category                                     
2014-01-02   low                       -          -           -  
             medium           43257622.0  1822719.0  30643745.0  
2014-01-03   low                       -          -           -  
             medium           38287706.0  1669229.0  31134795.0  
2014-01-06   high             68974359.0          -           -  
...                                  ...        ...         ...  
2019-08-21   low               8409548.0   708272.0  14982314.0  
             medium                    -          -           -  
2019-08-2

In [102]:
stacked.unstack(level=1)

Volume Category                               high      low       medium
Trading Date Ticker Previously a column axis                            
2014-01-02   AAPL   month                      NaN     1.00          NaN
                    year                       NaN  2014.00          NaN
                    day                        NaN     2.00          NaN
                    open                       NaN    79.38          NaN
                    close                      NaN    79.02          NaN
...                                            ...      ...          ...
2019-08-23   MSFT   open                       NaN      NaN       137.19
                    close                      NaN      NaN       133.39
                    high                       NaN      NaN       138.35
                    low                        NaN      NaN       132.80
                    volume                     NaN      NaN  38515386.00

[56840 rows x 3 columns]

### Creating Multilevel Columns Manually

In [103]:
tech.reset_index(inplace=True)

In [104]:
tech.head()

,Trading Date,Volume Category,Ticker,month,year,day,open,close,high,low,volume
0,2014-01-02,low,AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851
1,2014-01-02,medium,AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246
2,2014-01-02,medium,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622
3,2014-01-02,medium,GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719
4,2014-01-02,medium,MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745


In [105]:
# we want 2 levels MultiIndex columns with 
# - Trading volume (high, low) and Ticker(MSFT, AMZN),
# - 10 random closing prices for these combinations.

In [107]:
cols = pd.MultiIndex.from_product([['low','high'], ['MSFT', 'AMZN']], names=['Volume Category', 'Ticker'])

In [108]:
cols

MultiIndex([( 'low', 'MSFT'),
            ( 'low', 'AMZN'),
            ('high', 'MSFT'),
            ('high', 'AMZN')],
           names=['Volume Category', 'Ticker'])

In [109]:
low = tech['Volume Category'] == 'low'
high = tech['Volume Category'] == 'high'
msft = tech.Ticker == 'MSFT'
amzn = tech.Ticker == 'AMZN'

In [113]:
data = [
    tech[low & msft].close.sample(10).values, 
    tech[low & amzn].close.sample(10).values, 
    tech[high & msft].close.sample(10).values, 
    tech[high & amzn].close.sample(10).values
]

In [114]:
data = [[*i] for i in zip(*data)]

In [115]:
data

[[68.77, 1006.34, 42.88, 502.13],
 [138.79, 673.26, 40.16, 364.47],
 [64.93, 431.02, 47.87, 1371.99],
 [76.29, 335.13, 49.83, 1442.84],
 [47.45, 841.71, 105.91, 1390.0],
 [41.11, 401.01, 46.1, 1530.42],
 [53.93, 321.82, 46.13, 1429.95],
 [73.16, 559.44, 50.99, 1817.27],
 [57.6, 371.92, 53.49, 337.15],
 [83.93, 384.61, 50.79, 1598.01]]

In [116]:
pd.DataFrame(data= data, columns = cols)

Volume Category     low             high         
Ticker             MSFT     AMZN    MSFT     AMZN
0                 68.77  1006.34   42.88   502.13
1                138.79   673.26   40.16   364.47
2                 64.93   431.02   47.87  1371.99
3                 76.29   335.13   49.83  1442.84
4                 47.45   841.71  105.91  1390.00
5                 41.11   401.01   46.10  1530.42
6                 53.93   321.82   46.13  1429.95
7                 73.16   559.44   50.99  1817.27
8                 57.60   371.92   53.49   337.15
9                 83.93   384.61   50.79  1598.01

### An easier way: transpose()

In [118]:
tech.set_index(['Trading Date', 'Volume Category']).transpose()

Trading Date    2014-01-02                                       2014-01-03  \
Volume Category        low   medium    medium   medium    medium        low   
Ticker                AAPL     AMZN        FB    GOOGL      MSFT       AAPL   
month                    1        1         1        1         1          1   
year                  2014     2014      2014     2014      2014       2014   
day                      2        2         2        2         2          3   
open                 79.38    398.8     54.86   557.73     37.35       79.0   
close                79.02   397.97     54.71   556.56     37.16      77.28   
high                 79.58   399.36     55.22   558.88      37.4       79.1   
low                  78.86   394.02     54.19   554.13      37.1       77.2   
volume             8398851  2140246  43257622  1822719  30643745   14043410   

Trading Date                                           ... 2019-08-22  \
Volume Category   medium    medium   medium    medium  ...        low   
Ticker              AMZN        FB    GOOGL      MSFT  ...         FB   
month                  1         1        1         1  ...          8   
year                2014      2014     2014      2014  ...       2019   
day                    3         3        3         3  ...         22   
open              398.29      55.0    557.5      37.2  ...     183.43   
close             396.44     54.56    552.5     36.91  ...     182.04   
high              402.71     55.65   558.47     37.22  ...     184.11   
low               396.22     54.53   552.47      36.6  ...     179.91   
volume           2213512  38287706  1669229  31134795  ...   10829509   

Trading Date                                          2019-08-23           \
Volume Category      low    medium   medium    medium     medium   medium   
Ticker             GOOGL      AAPL     AMZN      MSFT       AAPL     AMZN   
month                  8         8        8         8          8        8   
year                2019      2019     2019      2019       2019     2019   
day                   22        22       22        22         23       23   
open              1193.8    213.19   1828.0    138.66     209.43  1793.03   
close            1191.52    212.46   1805.6    137.78     202.64  1749.62   
high             1198.78    214.44  1829.41     139.2     212.05   1804.9   
low              1178.91    210.75   1800.1    136.29      201.0  1745.23   
volume            867915  22267819  2658388  18559088   46882843  5277898   

Trading Date                                  
Volume Category    medium   medium    medium  
Ticker                 FB    GOOGL      MSFT  
month                   8        8         8  
year                 2019     2019      2019  
day                    23       23        23  
open               180.84  1185.17    137.19  
close              177.75  1153.58    133.39  
high               183.13  1195.67    138.35  
low                176.66   1150.0     132.8  
volume           17331221  1813141  38515386  

[9 rows x 7105 columns]

### Panels vs MultiIndex

In [121]:
# panels are deprecated since pd v0.22 
# Prefer MultiIndex for new projects 
# doc is still available for panels
# might be used in some legacy codes

In [122]:
# Challenge

In [127]:
# 1. Change the index of tech df to 3 level MultiIndex containing year, month, and day on levels 0, 1 and 2 respectively.
# name it tech_df3
tech_df3 = tech.set_index(['year', 'month', 'day']).sort_index()
tech_df3

Trading Date Volume Category Ticker     open    close     high  \
year month day                                                                  
2014 1     2     2014-01-02             low   AAPL    79.38    79.02    79.58   
           2     2014-01-02          medium   AMZN   398.80   397.97   399.36   
           2     2014-01-02          medium     FB    54.86    54.71    55.22   
           2     2014-01-02          medium  GOOGL   557.73   556.56   558.88   
           2     2014-01-02          medium   MSFT    37.35    37.16    37.40   
...                     ...             ...    ...      ...      ...      ...   
2019 8     23    2019-08-23          medium   AAPL   209.43   202.64   212.05   
           23    2019-08-23          medium   AMZN  1793.03  1749.62  1804.90   
           23    2019-08-23          medium     FB   180.84   177.75   183.13   
           23    2019-08-23          medium  GOOGL  1185.17  1153.58  1195.67   
           23    2019-08-23          medium   MSFT   137.19   133.39   138.35   

                    low    volume  
year month day                     
2014 1     2      78.86   8398851  
           2     394.02   2140246  
           2      54.19  43257622  
           2     554.13   1822719  
           2      37.10  30643745  
...                 ...       ...  
2019 8     23    201.00  46882843  
           23   1745.23   5277898  
           23    176.66  17331221  
           23   1150.00   1813141  
           23    132.80  38515386  

[7105 rows x 8 columns]

In [134]:
# 2. from tech_df3 dataset, select all the trading days from 2019.
# in addition, add existing column index as 4th level of MultiIndex and assign this series to tech_series.
tech_series = tech_df3.loc[2019].stack()

In [135]:
tech_series

month  day                 
1      2    Trading Date       2019-01-02
            Volume Category        medium
            Ticker                   AAPL
            open                   154.89
            close                  157.92
                                  ...    
8      23   open                   137.19
            close                  133.39
            high                   138.35
            low                     132.8
            volume               38515386
Length: 6520, dtype: object

In [141]:
# 3. from tech_series, find average close price and it's std deviation.
tech_series.xs('close', level=2).mean(), tech_series.xs('close', level=2).std()

(686.3976196319019, 677.2268499713305)